<a href="https://colab.research.google.com/github/amittal27/course-v3/blob/master/planet_experimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

### **Configure data**

In [18]:
path = Path.cwd()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/content/planet')

### **Multiclassification**

In [19]:
# read the csv file using the pandas library (popular way of dealing with tabular data in python), print the first five rows
df = pd.read_csv(path/'train_v2.csv')
df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [20]:
# data augmentation
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0) # do not want warp on the satellite images

In [21]:
# ensure we have the same validation set each time
np.random.seed(42)
# get images, get labels
src = (ImageList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .split_by_rand_pct(0.2) # set aside 20% of the training set (the current set of images) for the validation set
       .label_from_df(label_delim=' '))

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [17]:
# apply transforms, construct dataset
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats)) # use databunch to bind training and validation datasets

FileNotFoundError: ignored